In [1]:
from aqlm import QuantizedLinear
from aqlm.utils import _dequantize_weight, unpack_int_data

In [2]:
SIZE = 4096

layer = QuantizedLinear(
    in_features=SIZE,
    out_features=SIZE * 3,
    in_group_size=8,
    out_group_size=1,
    num_codebooks=2,
    nbits_per_codebook=8,
    bias=False,
)

In [3]:
reference_weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8),
    layer.codebooks,
    layer.scales,
)

In [4]:
import torch

input = torch.rand((1, 1, SIZE))

In [5]:
import torch
torch.ops.load_library("/Users/blacksamorez/reps/AQLM/inference_lib/src/aqlm/inference_kernels/cmake-out/libaqlm.dylib")

reference = input @ reference_weight.T
test = torch.ops.aqlm.code2x8_lut_matmat(
    input,
    torch.permute(layer.codes, (1, 0, 2)).contiguous(),
    layer.codebooks,
    layer.scales,
    layer.bias,
)

torch.testing.assert_close(
    test,
    reference,
    atol=0.01,
    rtol=1e-3,
)

In [6]:
%%time

for i in range(10):
    input @ reference_weight.T

CPU times: user 89 ms, sys: 32.9 ms, total: 122 ms
Wall time: 22.3 ms


In [7]:
%%time

for i in range(10):
    torch.ops.aqlm.code2x8_lut_matmat(
        input,
        torch.permute(layer.codes, (1, 0, 2)).contiguous(),
        layer.codebooks,
        layer.scales,
        layer.bias,
    )

CPU times: user 1.33 s, sys: 466 ms, total: 1.8 s
Wall time: 329 ms
